In [177]:
import pandas as pd
import numpy as np
import random
import json
from enum import Enum
from pathlib import Path

In [178]:
df = pd.read_csv("data/campusplan_evaluation.csv")

In [179]:
# fake building names
fake_lecture_halls = ["Einstein-Hörsaal", "Curie-Hörsaal", "Newton-Hörsaal", "Hawking-Hörsaal", "Galileo-Hörsaal", "Darwin-Hörsaal", "Tesla-Hörsaal", "Feynman-Hörsaal", "Planck-Hörsaal", "Bohr-Hörsaal", "Faraday-Hörsaal", "Mendel-Hörsaal", "Hubble-Hörsaal", "Turing-Hörsaal", "Rutherford-Hörsaal", "Archimedes-Hörsaal", "Sanger-Hörsaal", "von Neumann-Hörsaal", "Franklin-Hörsaal"]
fake_buildings = ["60.03", "60.20", "60.41", "60.19", "60.12", "20.61", "20.65", "20.91", "10.14", "10.74", "10.28", "10.35", "30.09", "30.26", "30.52", "40.20", "40.35", "40.61", "50.09", "50.78", "70.02"]

In [180]:
class Turns(Enum):
    SINGLE = 1
    MULTI = 2
    
def asemble_prompt(input_data, response_data, prompt, response):
    prompt = {"input_data" : input_data, "response_data" : response_data, "prompt" : prompt, "response" : response}
    return prompt

def build_prompt_dict(turns, prompts):
    prompt_dict = {"num_turns" : turns, "prompts" : prompts}
    return prompt_dict

def json_dump(turns, prompts, name):
    if turns == Turns.SINGLE:
        path = Path("data/evaluation/single_turn")
    elif turns == Turns.MULTI:
        path = Path("data/evaluation/multi_turn")

    with open(path / f"{name}.json", 'w') as f:
        json.dump(prompts, f, indent=4)

## Retrieve Static Information

In [181]:
# building location
prompts = []
for index, row in df.iterrows():
    building_id = row['title']
    address = row['rev_display_name']
    
    prompt = f"Wo ist Gebäude {building_id}?"
    
    if str(address) == "nan":
        response = "Ich kenne die Adresse von diesem Gebäude leider nicht."
    else:
        response = f"Das Gebäude hat die Adresse {address}."

    prompt = asemble_prompt(building_id, address, prompt, response)
    prompt_dict = build_prompt_dict(1, [prompt])
    prompts.append(prompt_dict)

for index, row in df.loc[df['rev_name'].notnull()].iterrows():
    building_id = row['rev_name']
    address = row['rev_display_name']
    
    prompt = f"Wo ist Gebäude {building_id}?"
    
    if str(address) == "nan":
        response = "Ich kenne die Adresse von diesem Gebäude leider nicht."
    else:
        response = f"Das Gebäude hat die Adresse {address}."

    prompt = asemble_prompt(building_id, address, prompt, response)
    prompt_dict = build_prompt_dict(1, [prompt])
    prompts.append(prompt_dict)
    
json_dump(Turns.SINGLE, prompts, "building_location")

# fake building location
prompts = []
for fake_hall in fake_lecture_halls:
    prompt = f"Wo ist Gebäude {fake_hall}?"
    response = "Zu diesem Gebäude habe ich leider keine Informationen."

    prompt = asemble_prompt(fake_hall, None, prompt, response)
    prompt_dict = build_prompt_dict(1, [prompt])
    prompts.append(prompt_dict)

for fake_building in fake_buildings:
    prompt = f"Wo ist Gebäude {fake_building}?"
    response = "Zu diesem Gebäude habe ich leider keine Informationen."

    prompt = asemble_prompt(fake_building, None, prompt, response)
    prompt_dict = build_prompt_dict(1, [prompt])
    prompts.append(prompt_dict)
    
json_dump(Turns.SINGLE, prompts, "building_location_fake")

In [182]:
# building category
prompts = []
for index, row in df.iterrows():
    building_id = row['title']
    category = row['category']
    
    prompt = f"Was für ein Gebäude ist {building_id}?"

    if str(category) == "nan":
        response = "Ich habe leider keine Informationen dazu, um was für ein Gebäude es sich handelt."
    else:
        response = f"Das Gebäude ist ein {category}."

    prompt = asemble_prompt(building_id, category, prompt, response)
    prompt_dict = build_prompt_dict(1, [prompt])
    prompts.append(prompt_dict)
    
json_dump(Turns.SINGLE, prompts, "building_category")

# fake building category
prompts = []
for fake_hall in fake_lecture_halls:
    prompt = f"Was für ein Gebäude ist {fake_hall}?"
    response = "Zu diesem Gebäude habe ich leider keine Informationen."

    prompt = asemble_prompt(fake_hall, None, prompt, response)
    prompt_dict = build_prompt_dict(1, [prompt])
    prompts.append(prompt_dict)

for fake_building in fake_buildings:
    prompt = f"Was für ein Gebäude ist {fake_building}?"
    response = "Zu diesem Gebäude habe ich leider keine Informationen."

    prompt = asemble_prompt(fake_building, None, prompt, response)
    prompt_dict = build_prompt_dict(1, [prompt])
    prompts.append(prompt_dict)
    
json_dump(Turns.SINGLE, prompts, "building_category_fake")

In [183]:
# building type
prompts = []
for index, row in df.iterrows():
    building_id = row['title']
    building_type = row['rev_type']
    
    prompt = f"Was befindet sich in Gebäude {building_id}?"

    if str(building_type) == "nan":
        response = "Ich habe keine Informationen dazu, was sich in dem Gebäude befindet."
    else:
        response = f"In dem Gbäude befindet sich {building_type}."

    prompt = asemble_prompt(building_id, building_type, prompt, response)
    prompt_dict = build_prompt_dict(1, [prompt])
    prompts.append(prompt_dict)
    
json_dump(Turns.SINGLE, prompts, "building_type")

# fake building type
prompts = []
for fake_hall in fake_lecture_halls:
    prompt = f"Was bfindet sich in Gebäude {fake_hall}?"
    response = "Zu diesem Gebäude habe ich leider keine Informationen."

    prompt = asemble_prompt(fake_hall, None, prompt, response)
    prompt_dict = build_prompt_dict(1, [prompt])
    prompts.append(prompt_dict)

for fake_building in fake_buildings:
    prompt = f"Was bfindet sich in Gebäude {fake_building}?"
    response = "Zu diesem Gebäude habe ich leider keine Informationen."

    prompt = asemble_prompt(fake_building, None, prompt, response)
    prompt_dict = build_prompt_dict(1, [prompt])
    prompts.append(prompt_dict)
    
json_dump(Turns.SINGLE, prompts, "building_type_fake")

In [184]:
# building opening hours
prompts = []
for index, row in df.iterrows():
    building_id = row['title']
    opening_hours = row['det_opening_hours']
    
    prompt = f"Was sind die Öffnungszeiten von Gebäude {building_id}?"

    if str(opening_hours) == "nan":
        response = "Für dieses Gebäude kenne ich leider keine Öffnungszeiten."
    else:
        response = f"Das Gebäude hat an {opening_hours} geöffnet."

    prompt = asemble_prompt(building_id, opening_hours, prompt, response)
    prompt_dict = build_prompt_dict(1, [prompt])
    prompts.append(prompt_dict)

for index, row in df.loc[df['rev_name'].notnull()].iterrows():
    building_id = row['rev_name']
    opening_hours = row['det_opening_hours']
    
    prompt = f"Was sind die Öffnungszeiten von Gebäude {building_id}?"

    if str(opening_hours) == "nan":
        response = "Für dieses Gebäude kenne ich leider keine Öffnungszeiten."
    else:
        response = f"Das Gebäude hat an {opening_hours} geöffnet."

    prompt = asemble_prompt(building_id, opening_hours, prompt, response)
    prompt_dict = build_prompt_dict(1, [prompt])
    prompts.append(prompt_dict)
    
json_dump(Turns.SINGLE, prompts, "opening_hours")

# fake building opening hours
prompts = []
for fake_hall in fake_lecture_halls:
    prompt = f"Was sind die Öffnungszeiten von Gebäude {fake_hall}?"
    response = "Zu diesem Gebäude habe ich leider keine Informationen."

    prompt = asemble_prompt(fake_hall, None, prompt, response)
    prompt_dict = build_prompt_dict(1, [prompt])
    prompts.append(prompt_dict)

for fake_building in fake_buildings:
    prompt = f"Was sind die Öffnungszeiten von Gebäude {fake_building}?"
    response = "Zu diesem Gebäude habe ich leider keine Informationen."

    prompt = asemble_prompt(fake_building, None, prompt, response)
    prompt_dict = build_prompt_dict(1, [prompt])
    prompts.append(prompt_dict)
    
json_dump(Turns.SINGLE, prompts, "opening_hours_fake")

In [185]:
# building wheelchair accessible
prompts = []
for index, row in df.iterrows():
    building_id = row['title']
    accessible = row['det_wheelchair']
    description = row['det_wheelchair:description']
    
    prompt = f"Ist Gebäude {building_id} rollstuhlgerecht?"

    if str(accessible) == "nan":
        response = "Zu diesem Gebäude habe ich leider keine Informationen zur Rollstuhgerechtikgeit."
    elif str(accessible) == "no":
        response = "Nein, das Gebäude ist nicht rollstuhlgerecht."
    elif str(accessible) == "limited":
        response = "Das Gebäude ist teilweise rollstuhlgerecht."
    elif str(accessible) == "yes":
        if str(description) == "nan":
            response = "Ja, das Gebäude ist rollstuhlgerecht."
        else:
            response = f"Ja, das Gebäude ist rollstuhlgerecht. Es gibt {description}."
            

    prompt = asemble_prompt(building_id, f"{accessible}, {description}", prompt, response)
    prompt_dict = build_prompt_dict(1, [prompt])
    prompts.append(prompt_dict)

for index, row in df.loc[df['rev_name'].notnull()].iterrows():
    building_id = row['rev_name']
    accessible = row['det_wheelchair']
    description = row['det_wheelchair:description']
    
    prompt = f"Ist Gebäude {building_id} rollstuhlgerecht?"

    if str(accessible) == "nan":
        response = "Zu diesem Gebäude habe ich leider keine Informationen zur Rollstuhgerechtikgeit."
    elif str(accessible) == "no":
        response = "Nein, das Gebäude ist nicht rollstuhlgerecht."
    elif str(accessible) == "limited":
        response = "Das Gebäude ist teilweise rollstuhlgerecht."
    elif str(accessible) == "yes":
        if str(description) == "nan":
            response = "Ja, das Gebäude ist rollstuhlgerecht."
        else:
            response = f"Ja, das Gebäude ist rollstuhlgerecht. Es gibt {description}."
            

    prompt = asemble_prompt(building_id, f"{accessible}, {description}", prompt, response)
    prompt_dict = build_prompt_dict(1, [prompt])
    prompts.append(prompt_dict)
    
json_dump(Turns.SINGLE, prompts, "wheelchair_accessible")

# fake building wheelchair accessible
prompts = []
for fake_hall in fake_lecture_halls:
    prompt = f"Ist Gebäude {fake_hall} rollstuhlgerecht?"
    response = "Zu diesem Gebäude habe ich leider keine Informationen."

    prompt = asemble_prompt(fake_hall, None, prompt, response)
    prompt_dict = build_prompt_dict(1, [prompt])
    prompts.append(prompt_dict)

for fake_building in fake_buildings:
    prompt = f"Ist Gebäude {fake_building} rollstuhlgerecht?"
    response = "Zu diesem Gebäude habe ich leider keine Informationen."

    prompt = asemble_prompt(fake_building, None, prompt, response)
    prompt_dict = build_prompt_dict(1, [prompt])
    prompts.append(prompt_dict)
    
json_dump(Turns.SINGLE, prompts, "wheelchair_accessible_fake")